In [9]:
import timm
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [ ]:
device = "cude" if torch.cuda.is_available() else "cpu"

In [10]:
def run_model_one_epoch(model, data_loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0

    for step, (inputs, labels) in tqdm(enumerate(data_loader), total=len(data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)

        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()

    return total_loss / (step + 1)

In [11]:
# Get pretrained model
model = timm.create_model('resnet50', pretrained=True)

# Preprocessing transform for model
config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [12]:
# Training parameters
batch_size = 128

# Imagenette dataset
train_dataset = torchvision.datasets.ImageFolder('imagenette2-320/train/', transform=transform)
val_dataset = torchvision.datasets.ImageFolder('imagenette2-320/val', transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, num_workers=5)
val_loader = DataLoader(val_dataset, batch_size, pin_memory=True, num_workers=5)

# Criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [13]:
from sparsezoo import search_models
from sparseml.pytorch.optim import ScheduledModifierManager

# zoo_model = search_models(
#     domain="cv",
#     sub_domain="classification",
#     architecture="resnet_v1",
#     sub_architecture="50",
#     framework="pytorch",
#     repo="sparseml",
#     dataset="imagenette",
#     sparse_name="pruned",
# )[0]  # unwrap search result
# recipe_path = zoo_model.recipes.default.path
recipe_path = 'recipe.md'
print(f"Using recipe: {recipe_path}")

manager = ScheduledModifierManager.from_yaml(recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

Using recipe: recipe.md


In [14]:
epoch = manager.min_epochs
while epoch < manager.max_epochs:
    train_loss = run_model_one_epoch(model, train_loader, criterion, 'cpu', train=True, optimizer=optimizer)
    epoch += 1


  5%|▌         | 4/74 [01:32<26:49, 23.00s/it]

: 

: 

In [24]:
[k for k,v in model.named_parameters()]

['conv1.weight',
 'bn1.weight',
 'bn1.bias',
 'layer1.0.conv1.weight',
 'layer1.0.bn1.weight',
 'layer1.0.bn1.bias',
 'layer1.0.conv2.weight',
 'layer1.0.bn2.weight',
 'layer1.0.bn2.bias',
 'layer1.0.conv3.weight',
 'layer1.0.bn3.weight',
 'layer1.0.bn3.bias',
 'layer1.0.downsample.0.weight',
 'layer1.0.downsample.1.weight',
 'layer1.0.downsample.1.bias',
 'layer1.1.conv1.weight',
 'layer1.1.bn1.weight',
 'layer1.1.bn1.bias',
 'layer1.1.conv2.weight',
 'layer1.1.bn2.weight',
 'layer1.1.bn2.bias',
 'layer1.1.conv3.weight',
 'layer1.1.bn3.weight',
 'layer1.1.bn3.bias',
 'layer1.2.conv1.weight',
 'layer1.2.bn1.weight',
 'layer1.2.bn1.bias',
 'layer1.2.conv2.weight',
 'layer1.2.bn2.weight',
 'layer1.2.bn2.bias',
 'layer1.2.conv3.weight',
 'layer1.2.bn3.weight',
 'layer1.2.bn3.bias',
 'layer2.0.conv1.weight',
 'layer2.0.bn1.weight',
 'layer2.0.bn1.bias',
 'layer2.0.conv2.weight',
 'layer2.0.bn2.weight',
 'layer2.0.bn2.bias',
 'layer2.0.conv3.weight',
 'layer2.0.bn3.weight',
 'layer2.0.bn3.b